## Notebook de processamento de arquivo bruto da matriz de integração

- Autor: Rodrigo Cunha (BT002004)
- Descrição: Processa arquivo bruto desformatado da matriz de integração, extraído do CITOP, e o formata para publicação
- Observação: Esse algoritmo ainda não foi validado pela Gerência de Controle, Estudos Tarifários e Tecnologia (GCETT-BT)

### Importação de bibliotecas

In [ ]:
import pandas as pd

### Carregamento do arquivo e pré-limpeza do arquivo
- Organiza as colunas do arquivo TXT
- Altera os nomes das colunas do arquivo TXT

In [ ]:
#Organização das colunas do arquivo TXT
cs = [(5, 13),
      (13, 53), 
      (53, 56),
      (56, 63),
      (63, 103),
      (103, 109),
      (109, 115),
      (115, 123),
      (123, 129),
      (129, 134)]

#Nomes das colunas do arquivo TXT
head = ['Linha_Sub_Origem', 'Linha_Sub_Origem_Nome', 'Linha_Origem_Grupo', 'Linha_Sub_Destino', 'Linha_Sub_Destino_Nome', 'Linha_Destino_Grupo', 'Tarifa_1_2', 'Tarifa_1_2_Valor', 'Tarifa_2_1', 'Tarifa_2_1_Valor']

#Nome do arquivo TXT
file = 'Matriz - 28.07.22.TXT'

#Leitura do arquivo TXT
df_matriz_integracao = pd.read_fwf(file, encoding='Windows-1252', colspecs=cs, names=head, skipinitialspace=True, skiprows=3)
df_matriz_integracao

### Limpeza do arquivo
- Remove linhas em branco
- Remove linhas repetidas
- Reinicia o índice

- Remove linhas em branco

In [ ]:
df_matriz_integracao.dropna(subset=['Linha_Sub_Origem'], inplace=True)
df_matriz_integracao

- Remove linhas repetidas
- Reinicia o índice

In [ ]:
remove_data = ['Legenda', 'TarID1', 'TarID2', 'CITCE', 'Linha Or']
df_matriz_integracao = df_matriz_integracao[~df_matriz_integracao['Linha_Sub_Origem'].str.contains('|'.join(remove_data))]
df_matriz_integracao = df_matriz_integracao.reset_index(drop=True)
df_matriz_integracao

### Tratamento do arquivo
- Subdivide 'Linha_Sub' (linha+sub) em 'Linha' e 'Sublinha'
- Elimina 'Linha_Sub' (linha+sub)
- Ordena pelas linhas e sublinhas
- Elimina integrações repetidas
- Reinicia o índice

- Subdivide 'Linha_Sub' (linha+sub) em 'Linha' e 'Sublinha'

In [ ]:
df_matriz_integracao[['Linha_Origem', 'Sublinha_Origem']] = df_matriz_integracao.loc[:,'Linha_Sub_Origem'].str.split('-', n=2, expand=True)
df_matriz_integracao[['Linha_Destino', 'Sublinha_Destino']] = df_matriz_integracao.loc[:,'Linha_Sub_Destino'].str.split('-', n=2, expand=True)
df_matriz_integracao

- Elimina 'Linha_Sub'

In [ ]:
df_matriz_integracao = df_matriz_integracao.drop(columns=['Linha_Sub_Origem', 'Linha_Sub_Destino'])
df_matriz_integracao

- Ordena pelas linhas e sublinhas

In [ ]:
df_matriz_integracao = df_matriz_integracao.sort_values(by=['Linha_Origem', 'Sublinha_Origem', 'Linha_Destino', 'Sublinha_Destino'])
df_matriz_integracao

- Elimina integrações repetidas observando 'Tarifa_1_2', 'Tarifa_1_2_Valor', 'Tarifa_2_1', 'Tarifa_2_1_Valor', 'Linha_Origem', e 'Linha_Destino' (ignora sublinhas)
- Reinicia o índice

In [ ]:
df_matriz_integracao = df_matriz_integracao.drop_duplicates(subset=['Tarifa_1_2', 'Tarifa_1_2_Valor', 'Tarifa_2_1', 'Tarifa_2_1_Valor', 'Linha_Origem', 'Linha_Destino'])
df_matriz_integracao = df_matriz_integracao.reset_index(drop=True)
df_matriz_integracao

### Salva o arquivo

In [ ]:
df_matriz_integracao.to_csv('matriz_integracao_processada.csv', index=False, sep=';', encoding='Windows-1252')